In [110]:
import numpy as np
import os
import glob
import SimpleITK as sitk
from skimage import measure
import nibabel as nib
import itk
from itkwidgets import view
import seaborn as sns
from mitok.utils.mdicom import SERIES

def connected_domain(image, mask=True):
    cca = sitk.ConnectedComponentImageFilter()
    cca.SetFullyConnected(True)
    _input = sitk.GetImageFromArray(image.astype(np.uint8))
    output_ex = cca.Execute(_input)
    stats = sitk.LabelShapeStatisticsImageFilter()
    stats.Execute(output_ex)
    num_label = cca.GetObjectCount()
    num_list = [i for i in range(1, num_label+1)]
    area_list = []
    for l in range(1, num_label +1):
        area_list.append(stats.GetNumberOfPixels(l))
    num_list_sorted = sorted(num_list, key=lambda x: area_list[x-1])[::-1]
    largest_area = area_list[num_list_sorted[0] - 1]
    final_label_list = [num_list_sorted[0]]

    for idx, i in enumerate(num_list_sorted[1:]):
        if area_list[i-1] >= (largest_area//10):
            final_label_list.append(i)
        else:
            break
    output = sitk.GetArrayFromImage(output_ex)

    for one_label in num_list:
        if  one_label in final_label_list:
            continue
        x, y, z, w, h, d = stats.GetBoundingBox(one_label)
        one_mask = (output[z: z + d, y: y + h, x: x + w] != one_label)
        output[z: z + d, y: y + h, x: x + w] *= one_mask

    if mask:
        output = (output > 0).astype(np.uint8)
    else:
        output = ((output > 0)*255.).astype(np.uint8)
    return output

def connected_component(image):
    # 标记输入的3D图像
    label, num = measure.label(image, connectivity=2, return_num=True)
    if num < 1:
        return [], image
        
    # 获取对应的region对象
    region = measure.regionprops(label)
    # 获取每一块区域面积并排序
    num_list = [i for i in range(1, num+1)]
    area_list = [region[i-1].area for i in num_list]
    num_list_sorted = sorted(num_list, key=lambda x: area_list[x-1])[::-1]
    # 去除面积较小的连通域
    '''
    if len(num_list_sorted) > 3:
        # for i in range(3, len(num_list_sorted)):
        for i in num_list_sorted[3:]:
            # label[label==i] = 0
            label[region[i-1].slice][region[i-1].image] = 0
        num_list_sorted = num_list_sorted[:3]
    '''
    return num_list_sorted, label
   
def np_count(nparray, x):
    i = 0
    for n in nparray:
        if n == x:
            i += 1
    return i

def remove_mix_region(num, volume, out_data):
    after_num = []
    after_volume = []
    for i in range(len(num)):
        if volume[i] > 0:
            after_num.append(num[i])
            after_volume.append(volume[i])
        else:
            out_data[out_data==num[i]] = 0
    return after_num, after_volume, out_data


def show_data(path):
    print(path)
    img = sitk.ReadImage(path)
    data = sitk.GetArrayFromImage(img)
    print(data.shape)
    data = np.unique(data)
    print(data)
    
plaque_path = '/mnt/users/transformed_plaque' 
proofread_plaque_path = '/mnt/users/transformed_plaque_proofread' 
plaque_list = os.listdir(plaque_path)
for plaque in plaque_list:
    plaque_dir = os.path.join(plaque_path, plaque)
    series_list = os.listdir(plaque_dir)
    for series in series_list:
        series_dir = os.path.join(plaque_dir, series, 'mask_plaque.nii.gz')
        print(series_dir)
        data = sitk.ReadImage(series_dir)
        data = sitk.GetArrayFromImage(data)
        num, out_data = connected_component(data)
        if len(num) != 0:
            volume = []
            for i in range(len(num)):
                volume.append(np_count(out_data[np.nonzero(out_data)], num[i]))
            print('原斑块为', num, volume)
        else:
            print('原来无斑块')
            
        proofread_series_dir = os.path.join(proofread_plaque_path, plaque, series, 'mask_plaque.nii.gz')
        print(proofread_series_dir)
        new_data = sitk.ReadImage(proofread_series_dir)
        new_data = sitk.GetArrayFromImage(new_data)
        num, out_data = connected_component(new_data)
        if len(num) != 0:
            volume = []
            for i in range(len(num)):
                volume.append(np_count(out_data[np.nonzero(out_data)], num[i]))
            print('校对后的斑块为', num, volume)
        else:
            print('校对后无斑块')
        

/mnt/users/transformed_plaque/1073332/5F654861/mask_plaque.nii.gz
原斑块为 [1] [3275]
/mnt/users/transformed_plaque_proofread/1073332/5F654861/mask_plaque.nii.gz
校对后的斑块为 [1] [2901]
/mnt/users/transformed_plaque/1073332/5F653FE9/mask_plaque.nii.gz


KeyboardInterrupt: 

In [111]:
series_name = '1073318/AF7B89E9'
vessel_plaque_nii = nib.load('/mnt/users/transformed_bad_case/'+series_name+'/mask_vessel_plaque_proofread.nii.gz')
vessel_plaque = vessel_plaque_nii.get_fdata()
plque_nii = nib.load('/mnt/users/transformed_plaque/'+series_name+'/mask_plaque.nii.gz')
proofread_plque_nii = nib.load('/mnt/users/transformed_plaque_proofread/'+series_name+'/mask_plaque.nii.gz')
plaque = plque_nii.get_fdata()
proofread_plaque = proofread_plque_nii.get_fdata()
pts0 = np.stack(np.where(plaque>0), axis=1)
pts1 = np.stack(np.where(proofread_plaque>0), axis=1)
pts2 = np.stack(np.where(vessel_plaque>0), axis=1)
pts3 = np.stack(np.where(vessel_plaque==1), axis=1)
pts4 = np.stack(np.where(vessel_plaque==2), axis=1)
view(point_sets=[pts2, pts3, pts4])

Viewer(geometries=[], gradient_opacity=0.22, point_set_colors=array([[0.8392157 , 0.        , 0.        ],
   …

In [112]:
#series_name = '1022836/56847483'
origin_vessel_plque_nii = nib.load('/mnt/users/ffr_plaque_mask/'+series_name+'/mask_vessel_plaque.nii.gz')
origin_vessel_plque = origin_vessel_plque_nii.get_fdata()
pts5 = np.stack(np.where(origin_vessel_plque>0), axis=1)
pts6 = np.stack(np.where(origin_vessel_plque==1), axis=1)
pts7 = np.stack(np.where(origin_vessel_plque==2), axis=1)
view(point_sets=[pts5, pts6, pts7])

Viewer(geometries=[], gradient_opacity=0.22, point_set_colors=array([[0.8392157 , 0.        , 0.        ],
   …

In [113]:
view(point_sets=[pts0, pts1])

Viewer(geometries=[], gradient_opacity=0.22, point_set_colors=array([[0.8392157 , 0.        , 0.        ],
   …